# 0. Setup Paths

In [ ]:
import os

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# for path in paths.values():
#     if not os.path.exists(path):
#         if os.name == 'posix':
#            %mkdir -p {path}
#         if os.name == 'nt':
#            %mkdir {path}


# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

while running above it shows no module named numpy so installed it

In [ ]:
# %pip install protobuf==3.19.0

In [ ]:
# %pip install protobuf

In [ ]:
import os

# Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION environment variable
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
# %pip install pyyaml

In [ ]:
# %pip install tensorflow --upgrade

In [ ]:
# %pip install --upgrade setuptools

In [ ]:
# %pip install matplotlib

In [ ]:
import object_detection

In [ ]:
# !pip list

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 2. Create Label Map

In [ ]:
labels = [{'name':'licence', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

# OPTIONAL IF RUNNING ON COLAB
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
# %pip install pytz

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

# 4. Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=6000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
# !{command}

# 7. Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
# !{command}

# 8. Load Train Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Prevent GPU complete consumption
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try: 
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    except RunTimeError as e:
        print(e)

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-7')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect from an Image

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'Cars417.png')

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.7,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

#Apply OCR to Detection

In [ ]:
# import os
# import stat

# def check_permissions(path):
#     # Get the file permissions using os.stat()
#     permissions = stat.S_IMODE(os.stat(path).st_mode)
    
#     # Convert the permissions to octal representation
#     octal_permissions = oct(permissions)
    
#     return octal_permissions

# # Example usage
# file_path = 'C:\\ANPR3\\anprsys1\\Lib\\site-packages\\cv2\\cv2.pyd'
# permissions = check_permissions(file_path)
# print(f'Permissions of {file_path}: {permissions}')


In [ ]:
# import os

# # Specify the path to the 'cv2.pyd' file
# file_path = 'C:\\ANPR3\\anprsys1\\Lib\\site-packages\\cv2\\cv2.pyd'

# # Check the permissions for the file
# permissions = os.stat(file_path).st_mode

# # Print the permissions in octal format
# print(oct(permissions))

In [ ]:
# %pip install easyocr

In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
# %pip install pytorch

In [ ]:
# %pip install --upgrade Pillow

In [ ]:
# %pip install Pillow==9.5.0

In [ ]:
import easyocr

In [ ]:
import cv2

In [ ]:
detection_threshold = 0.7

In [ ]:
image = image_np_with_detections
scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
boxes = detections['detection_boxes'][:len(scores)]
classes = detections['detection_classes'][:len(scores)]

In [ ]:
width = image.shape[1]
height = image.shape[0]

In [ ]:
# Apply ROI filtering and OCR
for idx, box in enumerate(boxes):
    print(box)
    roi = box * [height, width, height, width]
    print(roi)
    region = image[int(roi[0]):int(roi[2]), int(roi[1]):int(roi[3])]
    reader = easyocr.Reader(['en'])
    ocr_result = reader.readtext(region)
    
    if ocr_result is not None:
        for result in ocr_result:
            print(np.sum(np.subtract(result[0][2], result[0][1])))
            print(result[1])
    
    plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))


In [ ]:

for result in ocr_result:
    print(np.sum(np.subtract(result[0][2],result[0][1])))
    print(result[1])

OCR filtering

In [ ]:
region_threshold = 0.06

In [ ]:
def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    
    plate = [] 
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate

In [ ]:
# filter_text(region, ocr_result, region_threshold)
# Assuming you have the necessary imports and the filter_text function is defined properly
# import numpy as np

# # Define and assign values to region and ocr_result (example data)
# region = "Some region"
# ocr_result = [
#     ([1, 2, 3], "Result 1"),
#     ([4, 5, 6], "Result 2"),
#     ([7, 8, 9], "Result 3")
# ]

# # Define the filter_text function (if not already defined)
# def filter_text(region, ocr_result, threshold):
#     # Your filtering logic here
#     pass

# # Call the filter_text function
# region_threshold = 0.5  # Replace this value with your desired threshold
filter_text(region, ocr_result, region_threshold)


BRING IT TOGETHER

In [ ]:
region_threshold = 0.6

In [ ]:
def ocr_it(image, detections, detection_threshold, region_threshold):
    
    # Scores, boxes and classes above threhold
    scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    # Full image dimensions
    width = image.shape[1]
    height = image.shape[0]
    
    # Apply ROI filtering and OCR
    for idx, box in enumerate(boxes):
        roi = box*[height, width, height, width]
        region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
        reader = easyocr.Reader(['en'])
        ocr_result = reader.readtext(region)
        
        text = filter_text(region, ocr_result, region_threshold)
        
        plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
        plt.show()
        print(text)
        return text, region

#---------------------------------------------------
# import csv
# import cv2
# import easyocr
# import re

# def filter_text(ocr_result, region_threshold):
#     # Check if ocr_result is empty
#     if not ocr_result:
#         return ""

#     # Implement your text filtering logic here
#     # For example, you can use regex to keep only alphanumeric characters
#     text = ocr_result[0][1]  # The OCR text is stored at index 1 in the first element of the ocr_result list
#     alphanumeric_text = re.sub(r'[^a-zA-Z0-9]', '', text)
#     return alphanumeric_text

# def ocr_it(image, detections, detection_threshold, region_threshold, output_csv_file):
#     # Scores, boxes, and classes above the threshold
#     scores = list(filter(lambda x: x > detection_threshold, detections['detection_scores']))
#     boxes = detections['detection_boxes'][:len(scores)]
#     classes = detections['detection_classes'][:len(scores)]

#     # Full image dimensions
#     width = image.shape[1]
#     height = image.shape[0]

#     # Initialize an empty list to store the filtered OCR results
#     ocr_results_filtered = []

#     # Apply ROI filtering and OCR
#     for idx, box in enumerate(boxes):
#         roi = box * [height, width, height, width]
#         region = image[int(roi[0]):int(roi[2]), int(roi[1]):int(roi[3])]
#         reader = easyocr.Reader(['en'])
#         ocr_result = reader.readtext(region)

#         filtered_text = filter_text(ocr_result, region_threshold)

#         # Append the filtered text to the list
#         ocr_results_filtered.append(filtered_text)

#         # Display the region and the filtered text
#         plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
#         plt.show()
#         print(filtered_text)

#     # Write the filtered OCR results to a CSV file
#     with open(output_csv_file, 'w', newline='') as csvfile:
#         csv_writer = csv.writer(csvfile)
#         for text in ocr_results_filtered:
#             csv_writer.writerow([text])

#     return ocr_results_filtered

# # Example usage:
# # Assuming you have 'image', 'detections', 'detection_threshold', and 'region_threshold' defined
# output_csv_file = 'filtered_ocr_results.csv'
# filtered_results = ocr_it(image, detections, detection_threshold, region_threshold, output_csv_file)




In [ ]:
# import pandas as pd
# import json

# # Step 1: Read the CSV file into a DataFrame
# csv_file_path = 'C:/ANPR3/realtimeresults.csv'
# df = pd.read_csv(csv_file_path)

# # Step 2: Load the JSON data into a Python dictionary
# json_file_path = 'path/to/your/jsonfile.json'
# with open(json_file_path, 'r') as f:
#     json_data = json.load(f)

# # Step 3-5: Iterate through the DataFrame and check for matches in the JSON
# for index, row in df.iterrows():
#     number_plate = row['Number Plate']
#     if number_plate in json_data:
#         # Number plate found in the JSON, extract and display info
#         info = json_data[number_plate]
#         name = info['name']
#         address = info['address']

#         # Display the info as a pop-up (you can use any GUI library for this)
#         print(f"Number Plate: {number_plate}")
#         print(f"Name: {name}")
#         print(f"Address: {address}")

In [ ]:
text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold)

save results

In [ ]:
import csv
import uuid

In [ ]:
'{}.jpg'.format(uuid.uuid1())

In [ ]:
def save_results(text, region, csv_filename, folder_path):
    img_name = '{}.jpg'.format(uuid.uuid1())
    
    cv2.imwrite(os.path.join(folder_path, img_name), region)
    
    with open(csv_filename, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow([img_name, text])

In [ ]:
region

In [ ]:
save_results(text, region, 'detection_results.csv', 'Detection_Images')


## CONVERTING TO FRAMES 

In [ ]:
# import os
# import cv2

# def extract_frames(input_video_path, output_folder, frame_interval=2):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     video_capture = cv2.VideoCapture(input_video_path)
#     frame_rate = video_capture.get(cv2.CAP_PROP_FPS)
#     interval_frames = int(frame_rate * frame_interval)

#     frame_count = 0
#     success, image = video_capture.read()
#     while success:
#         if frame_count % interval_frames == 0:
#             output_file = os.path.join(output_folder, f"frame_{frame_count}.jpg")
#             cv2.imwrite(output_file, image)

#         frame_count += 1
#         success, image = video_capture.read()

#     video_capture.release()

# if __name__ == "__main__":
#     input_video_path = 'C:/ANPR3/Tensorflow/workspace/images/test/License Plate Camera vs Non-LPR Security Camera (720p).mp4'
#     output_folder = 'C:/ANPR3/output_frames'
#     frame_interval = 1 # Change this value to set the desired interval in seconds

#     extract_frames(input_video_path, output_folder, frame_interval)


ocring the folder

In [ ]:
# import os
# import cv2
# import numpy as np
# import tensorflow as tf
# from object_detection.utils import label_map_util
# from object_detection.utils import visualization_utils as viz_utils
# import pytesseract

# def run_object_detection(image_np):
#     # Object detection code here
#     # ...

# def run_ocr(image_np_with_detections):
#     # OCR code here
#     # ...

# # Load the object detection model
# detect_fn = tf.saved_model.load('path/to/object_detection_model')

# # Load the label map and category index
# label_map_path = 'path/to/label_map.pbtxt'
# category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

# # Path to the folder containing the images
# image_folder = 'path/to/image/folder'

# # Path to save the results
# results_folder = 'path/to/results/folder'

# # Iterate over the images in the folder
# for image_file in os.listdir(image_folder):
#     if image_file.endswith('.png') or image_file.endswith('.jpg'):
#         # Read the image
#         image_path = os.path.join(image_folder, image_file)
#         image_np = cv2.imread(image_path)

#         # Run object detection on the image
#         image_np_with_detections = run_object_detection(image_np)

#         # Run OCR on the image with detections
#         text = run_ocr(image_np_with_detections)

#         # Save the text to a file
#         output_file = os.path.join(results_folder, f'{image_file}.txt')
#         with open(output_file, 'w') as f:
#             f.write(text)

#         print(f"Processed image: {image_file}")

# print("Task completed!")


# 10. Real Time Detections from your Webcam

In [ ]:
# !pip uninstall opencv-python-headless -y

In [ ]:
# import csv
# import json
# import tkinter as tk
# from tkinter import messagebox

# # Read the JSON file containing the mapping of plate text to information
# def read_json_file(json_file):
#     with open(json_file, 'r') as file:
#         data = json.load(file)
#     return data

# # Real-time CSV reading and processing function
# def read_real_time_csv(csv_file, json_file):
#     # Read the JSON file with plate text to information mapping
#     data = read_json_file(json_file)

#     # Initialize tkinter root window
#     root = tk.Tk()
#     root.withdraw()  # Hide the root window

#     # Continuously monitor the CSV file for new entries
#     while True:
#         with open(csv_file, 'r') as file:
#             reader = csv.reader(file)
#             print(reader)
#             # Skip the header row if it exists
#             next(reader, None)

#             # Process new entries in the CSV file
#             for row in reader:
#                 plate_text = row[0]  # Assuming plate text is in the first column
#                 # Check if the plate text exists in the JSON data
#                 if plate_text in data:
#                     info = data[plate_text]
#                     # Use 'info' to show a pop-up with the fetched information
#                     messagebox.showinfo("Number Plate Info", f"Plate: {plate_text}\nInfo: {info}")

# if __name__ == "_main_":
#     # Replace 'real_time.csv' with the path to your real-time CSV file
#     csv_file_path = 'C:/ANPR3/test.csv'
#     # Replace 'info_mapping.json' with the path to your JSON file
#     json_file_path = 'C:/ANPR3/data.json'
#     read_real_time_csv(csv_file_path, json_file_path)

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)
    
    try: 
        text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold)
        save_results(text, region, 'realtimeresults.csv', 'Detection_Images')
    except:
        pass

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

In [ ]:
#-------------------------------------------





# import cv2
# import numpy as np
# import tensorflow as tf
# from object_detection.utils import visualization_utils as viz_utils

# def process_video_file(path_to_video_file):
#     cap = cv2.VideoCapture(path_to_video_file)

#     def ocr_it(image, detections, detection_threshold, region_threshold):
#         # Add your OCR logic here
#         scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
#         boxes = detections['detection_boxes'][:len(scores)]
#         classes = detections['detection_classes'][:len(scores)]
    
#         # Full image dimensions
#         width = image.shape[1]
#         height = image.shape[0]
    
#     # Apply ROI filtering and OCR
#         for idx, box in enumerate(boxes):
#             roi = box*[height, width, height, width]
#             region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
#             reader = easyocr.Reader(['en'])
#             ocr_result = reader.readtext(region)
        
#             text = filter_text(region, ocr_result, region_threshold)
        
#             plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
#             plt.show()
#             print(text)
#             return text, region
    
#     def save_results(text, region, csv_filename, image_folder):
#         # Add your saving logic here
#         img_name = '{}.jpg'.format(uuid.uuid1())
    
#         cv2.imwrite(os.path.join(folder_path, img_name), region)
    
#         with open(csv_filename, mode='a', newline='') as f:
#             csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#             csv_writer.writerow([img_name, text])
    
#     while cap.isOpened():
#         ret, frame = cap.read()
        
#         if not ret:
#             break
        
#         image_np = np.array(frame)

#         input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
#         detections = detect_fn(input_tensor)

#         num_detections = int(detections.pop('num_detections'))
#         detections = {key: value[0, :num_detections].numpy()
#                       for key, value in detections.items()}
#         detections['num_detections'] = num_detections

#         detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

#         label_id_offset = 1
#         image_np_with_detections = image_np.copy()

#         viz_utils.visualize_boxes_and_labels_on_image_array(
#                     image_np_with_detections,
#                     detections['detection_boxes'],
#                     detections['detection_classes']+label_id_offset,
#                     detections['detection_scores'],
#                     category_index,
#                     use_normalized_coordinates=True,
#                     max_boxes_to_draw=5,
#                     min_score_thresh=.8,
#                     agnostic_mode=False)

#         try: 
#             text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold)
#             save_results(text, region, 'realtimeresults.csv', 'Detection_Images')
#         except:
#             pass

#         cv2.imshow('object_detection', cv2.resize(image_np_with_detections, (800, 600)))

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

# # Call the function and provide the path to the video file
# process_video_file('C:/ANPR3/Tensorflow/workspace/images/test/License Plate Camera vs Non-LPR Security Camera (720p).mp4')

In [ ]:
# !pip install IPython

In [ ]:
# import cv2
# from IPython.display import display, Image
# import PIL.Image

# # Function to read frames from video and display in Jupyter cell
# def display_video(video_path):
#     video_capture = cv2.VideoCapture(video_path)

#     try:
#         while True:
#             ret, frame = video_capture.read()

#             if not ret:
#                 # Break the loop when video ends
#                 break

#             # Convert the frame from BGR to RGB (OpenCV uses BGR by default)
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#             # Display the frame in Jupyter cell
#             img = PIL.Image.fromarray(frame_rgb)
#             display(img)

#     except KeyboardInterrupt:
#         # Interrupt the video loop if you press the interrupt key (e.g., Ctrl+C)
#         pass

#     # Release the video capture object
#     video_capture.release()

# # Replace 'video_path' with the path to your video file
# video_path = 'C:/ANPR3/Tensorflow/workspace/images/test/License Plate Camera vs Non-LPR Security Camera (720p).mp4'
# display_video(video_path)

EXTRACTING TEXT

In [ ]:
# import os
# import pytesseract
# import cv2
# import pandas as pd
# from IPython.display import clear_output
# from PIL import Image
# import time

# # Replace 'YOUR_IMAGE_FOLDER' with the path to your folder containing images.
# image_folder = 'C:/ANPR3/testdec'

# # Replace 'YOUR_EXCEL_FILE' with the desired output Excel file name.
# output_excel_file = 'C:/ANPR3/plate_text.xlsx'

# # Create an empty DataFrame to store the results
# columns = ['Image', 'Extracted Text']
# results_df = pd.DataFrame(columns=columns)

# def ocr_and_save_to_excel():
#     global results_df
#     accumulator = []  # Create a list to accumulate the DataFrames
    
#     while True:
#         for image_file in os.listdir(image_folder):
#             if image_file.endswith('.jpg') or image_file.endswith('.png'):
#                 image_path = os.path.join(image_folder, image_file)
#                 try:
#                     # Read the image using OpenCV
#                     img = cv2.imread(image_path)
#                     # Convert to RGB format (required by Tesseract)
#                     img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#                     # Perform OCR on the image
#                     extracted_text = pytesseract.image_to_string(img_rgb)

#                     # Create a new DataFrame with the extracted text
#                     new_df = pd.DataFrame({'Image': [image_file], 'Extracted Text': [extracted_text]})
#                     accumulator.append(new_df)  # Append the DataFrame to the list

#                     clear_output(wait=True)
#                     print(f"Processed image: {image_file}")

#                 except Exception as e:
#                     print(f"Error processing {image_file}: {e}")
        
#         # Concatenate all the DataFrames in the list into one big DataFrame
#         results_df = pd.concat(accumulator, ignore_index=True)
        
#         # Save the DataFrame to the Excel file
#         results_df.to_excel(output_excel_file, index=False)
        
#         # Clear the accumulator for the next iteration
#         accumulator.clear()
        
#         # Wait for 1 second before the next iteration
#         time.sleep(1)

# # Call the function to start the OCR process
# ocr_and_save_to_excel()


In [ ]:
# %pip install openpyxl

In [ ]:
# %pip install pytesseract

# 10. Freezing the Graph

In [ ]:
# FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
# command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [ ]:
# print(command)

In [ ]:
# !{command}

# 11. Conversion to TFJS

In [ ]:
# !pip install tensorflowjs

In [ ]:
# command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])

In [ ]:
# print(command)

In [ ]:
# !{command}

In [ ]:
# Test Code: https://github.com/nicknochnack/RealTimeSignLanguageDetectionwithTFJS

# 12. Conversion to TFLite

In [ ]:
# TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
# command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])

In [ ]:
# print(command)

In [ ]:
# !{command}

In [ ]:
# FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
# TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
# command = "tflite_convert \
# --saved_model_dir={} \
# --output_file={} \
# --input_shapes=1,300,300,3 \
# --input_arrays=normalized_input_image_tensor \
# --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
# --inference_type=FLOAT \
# --allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [ ]:
# print(command)

In [ ]:
# !{command}

# 13. Zip and Export Models 

In [ ]:
# !tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %pip install google.colab